### WalkSAT plus Data Analysis of Results, Daniel Espinosa 2024 UCSB ECE Strukov Group + FPGA copy


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator
import xml.etree.ElementTree as Xet

In [2]:
import random

def walkSAT(clauses, max_tries, max_flips, p):
    def evaluate_clause(clause, assignment):
        return any((var > 0 and assignment.get(abs(var), False)) or 
                   (var < 0 and not assignment.get(abs(var), False)) for var in clause)

    def get_unsatisfied_clauses(clauses, assignment):
        return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

    def get_variables(clauses):
        return set(abs(var) for clause in clauses for var in clause)

    def flip_variable(assignment, var):
        assignment[var] = not assignment[var]

    for _ in range(max_tries):
        variables = list(get_variables(clauses))
        assignment = {var: random.choice([True, False]) for var in variables}
        
        for _ in range(max_flips):
            unsatisfied = get_unsatisfied_clauses(clauses, assignment)
            if not unsatisfied:
                return assignment  # Found a satisfying assignment
            
            clause = random.choice(unsatisfied)
            if random.random() < p:
                # Flip a random variable from the clause
                var_to_flip = abs(random.choice(clause))
            else:
                # Flip a variable that minimizes the number of unsatisfied clauses if flipped
                break_counts = []
                for var in clause:
                    assignment[abs(var)] = not assignment[abs(var)]
                    break_counts.append((len(get_unsatisfied_clauses(clauses, assignment)), abs(var)))
                    assignment[abs(var)] = not assignment[abs(var)]  # Undo the flip

                min_break = min(break_counts, key=lambda x: x[0])
                vars_with_min_break = [var for break_count, var in break_counts if break_count == min_break[0]]
                var_to_flip = random.choice(vars_with_min_break)
            
            flip_variable(assignment, var_to_flip)

    return "FAIL"

# Example SAT CNF formula: (x1 or not x2) and (not x1 or x2)
# clauses = [[1, -2],[-1,2]]
# Example UNSAT CNF formula: 
clauses = [[1, 2],[-1,-2],[1,-2],[-1,2]]
# Parameters
max_tries = 100
max_flips = 1000
p = 0.3

# Running WalkSAT
result = walkSAT(clauses, max_tries, max_flips, p)
print("Satisfying assignment found:" if result != "FAIL" else "No satisfying assignment found within the given limits.", result)

No satisfying assignment found within the given limits. FAIL


Now, structuring this based on the PDF using binary values for everything

In [3]:
def parse_cnf_file(file_path):
    num_vars = 0
    num_clauses = 0
    clauses = []

    with open(file_path, 'r') as file:
        for line in file:
            # Strip leading and trailing whitespace
            line = line.strip()

            # Ignore comment lines
            if line.startswith('c'):
                continue
            
            # Process the problem line
            elif line.startswith('p'):
                parts = line.split()
                if len(parts) == 4 and parts[1] == 'cnf':
                    num_vars = int(parts[2])
                    num_clauses = int(parts[3])
            
            # Process clause lines
            else:
                # Convert numbers to integers and remove the trailing 0
                clause = [int(x) for x in line.split()[:-1]]
                clauses.append(clause)
    
    return num_vars, num_clauses, clauses

def random_assignment(cnf):
#    Assuming cnf.variables is an iterable of variable identifiers
    return {var: random.choice([1, 0]) for var in cnf.variables}

In [45]:
parse_cnf_file('mnci_n16_t3_0_3sat.cnf')

class CNF:
    def __init__(self, num_vars, num_clauses, clauses):
        self.num_vars = num_vars
        self.num_clauses = num_clauses
        self.clauses = clauses

    def __str__(self,verbose=False):
        description = f"CNF Formula with {self.num_vars} variables and {self.num_clauses} clauses:\n"
        if verbose: 
            for clause in self.clauses:
                description += " ".join(str(var) for var in clause) + " 0\n"
        return description

    def variables(self):
        return range(self.num_vars)
    
    def random_assignment(self):
        return [random.choice([0, 1]) for var in CNF.variables(self)]
    
    def variable_membership(self):
        # We are making this so we can quickly make a LUT to see where variables are
        # To update only parts of the CNF and not handle the whole thing at once.
        # Maps each variable to the clauses it appears in (by index)
        """
        Usage: cnf.variable_membership.get(literal_integer) gives us all the clause
        numbers that the literal is a part of. Once we have all the memberships
        we can encode them to a single binary number later.
        """
        membership = {var: [] for var in self.variables()}
        for clause_index, clause in enumerate(self.clauses):
            for var in clause:
                var = abs(var)  # Consider the variable, not its negation
                if var in membership:
                    membership[var].append(clause_index)
        return membership

    def add_clause(self, clause):
        self.clauses.append(clause)
        self.num_clauses += 1

In [46]:
outputs = parse_cnf_file('mnci_n16_t3_0_3sat.cnf')
cnf = CNF(outputs[0],outputs[1],outputs[2])

In [64]:
cnf.variable_membership().get(1)

[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 260, 261, 262, 263, 264, 265]

In [66]:
start = cnf.random_assignment() 

In [68]:
start[10]

0

In [72]:
cnf.clauses[0]

[17, 13]

In [83]:
membership_dictionary = cnf.variable_membership()
#[x in range(10), cnf.clauses[membership_dictionary.get(1)[x]]]

start = cnf.random_assignment() 
testclause = cnf.clauses[0]

def evaluate_clause(clause, assignment):
    return any((var > 0 and assignment[abs(var) - 1]) or 
               (var < 0 and not assignment[abs(var) - 1]) for var in clause)

def get_unsatisfied_clauses(clauses, assignment):
    return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

evaluate_clause(testclause, start)

False

In [ ]:

def evaluate_clause(clause, assignment):
    return any((var > 0 and assignment.get(abs(var), False)) or 
               (var < 0 and not assignment.get(abs(var), False)) for var in clause)
# ^ not correct, needs to be for a list and I wrote it for a dict, oops


def flip_variable(assignment, var):
    assignment[var] = not assignment[var]

def get_unsatisfied_clauses(clauses, assignment):
    return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

def binarywalkSAT(cnf: CNF, max_tries=100, max_flips=1000, p=0.3):
    start = cnf.random_assignment() 

    for _ in range(max_tries):
        assignment = start.copy() #1 bit vector assignment
        unsatisfied = get_unsatisfied_clauses(cnf.clauses, assignment) 

        for _ in range(max_flips):

            # unsat clause buffer

            if not unsatisfied:
                return assignment  # If all unsat are sat we are done
            
            clause = random.choice(unsatisfied)


            if random.random() < p:
                var_to_flip = abs(random.choice(clause))
            else:
                break_counts = []
                for var in clause:
                    flip_variable(assignment, abs(var))
                    break_counts.append((len(get_unsatisfied_clauses(cnf.clauses, assignment)), abs(var)))
                    flip_variable(assignment, abs(var))  # Undo the flip

                min_break = min(break_counts, key=lambda x: x[0])
                var_to_flip = random.choice([var for _, var in break_counts if _ == min_break[0]])
            
            flip_variable(assignment, var_to_flip)

    return "FAIL"
